# VacationPy
---

## Code to Import Libraries and Load the Weather and Coordinates Data

In [34]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [35]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Resources/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount pearl,47.5166,-52.7813,278.79,71,75,11.32,CA,1698624783
1,1,hermanus,-34.4187,19.2345,286.42,64,0,10.33,ZA,1698624534
2,2,kenai,60.5544,-151.2583,279.20,80,100,8.75,US,1698624824
3,3,port-aux-francais,-49.3500,70.2167,275.07,70,100,11.06,TF,1698624629
4,4,vilyuysk,63.7553,121.6247,258.71,97,61,2.24,RU,1698624824


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [57]:
# Factorize the 'City' column to get unique integer codes
city_data_df['city_codes'] = pd.factorize(city_data_df['City'])[0]

# Normalize these codes to fall between 0 and 1
max_code = city_data_df['city_codes'].max()
city_data_df['city_norm'] = city_data_df['city_codes'] / max_code

# Configure the map plot
map_plot_1 = city_data_df.hvplot(
    x="Lng",
    y="Lat",
    hover_cols=["City"],
    geo=True,
    tiles="OSM",
    title="Global City Locations by Humidity Levels",
    frame_width=500,
    frame_height=500,
    kind='points',
    color='city_norm',
    cmap='tab20',           
    size='Humidity',
    alpha=0.6
)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (city_norm,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.dropna(inplace=True)

# Drop any rows with null values


# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount pearl,47.5166,-52.7813,278.79,71,75,11.32,CA,1698624783
1,1,hermanus,-34.4187,19.2345,286.42,64,0,10.33,ZA,1698624534
2,2,kenai,60.5544,-151.2583,279.20,80,100,8.75,US,1698624824
3,3,port-aux-francais,-49.3500,70.2167,275.07,70,100,11.06,TF,1698624629
4,4,vilyuysk,63.7553,121.6247,258.71,97,61,2.24,RU,1698624824


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mount pearl,CA,47.5166,-52.7813,71,
1,hermanus,ZA,-34.4187,19.2345,64,
2,kenai,US,60.5544,-151.2583,80,
3,port-aux-francais,TF,-49.3500,70.2167,70,
4,vilyuysk,RU,63.7553,121.6247,97,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000 # meters
limit = 1

params = {
    "radius": radius,
    "type": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    # Convert the API response to JSON format
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mount pearl - nearest hotel: Sandman Signature St. John's
hermanus - nearest hotel: Aloe guest house
kenai - nearest hotel: Beluga Lookout Hotel
port-aux-francais - nearest hotel: Keravel
vilyuysk - nearest hotel: Мария
puerto natales - nearest hotel: Dorotea Patagonia Hostel
chonchi - nearest hotel: Hotel Huildín
port douglas - nearest hotel: Mantra Aqueous Port Douglas
betio village - nearest hotel: Seaman Hostel
olonkinbyen - nearest hotel: No hotel found
mae hong son - nearest hotel: The Like View
utrik - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
atar - nearest hotel: فندق سكليل
mil'kovo - nearest hotel: Дрозды
vilhena - nearest hotel: No hotel found
tsengel - nearest hotel: No hotel found
barra de santo antonio - nearest hotel: Resort Fazenda Fiore
yasnyy - nearest hotel: No hotel found
sainte-menehould - nearest hotel: L'Orée du Bois
waitangi - nearest hotel: Hotel C

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mount pearl,CA,47.5166,-52.7813,71,Sandman Signature St. John's
1,hermanus,ZA,-34.4187,19.2345,64,Aloe guest house
2,kenai,US,60.5544,-151.2583,80,Beluga Lookout Hotel
3,port-aux-francais,TF,-49.3500,70.2167,70,Keravel
4,vilyuysk,RU,63.7553,121.6247,97,Мария
...,...,...,...,...,...,...
574,mata-utu,WF,-13.2816,-176.1745,74,Gîte Oceania
575,khandud,AF,36.9513,72.3180,64,Yamg Homestay
576,pukchil-lodongjagu,KP,40.2019,125.7483,70,No hotel found
577,zaozersk,RU,69.4005,32.4476,95,Северное сияние


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [62]:
%%capture --no-display

# Factorize the 'City' column to get unique integer codes
hotel_df['city_codes'] = pd.factorize(hotel_df['City'])[0]

# Normalize these codes to fall between 0 and 1
max_code = hotel_df['city_codes'].max()
hotel_df['city_norm'] = hotel_df['city_codes'] / max_code

# Configure the map plot
map_plot_2 = hotel_df.hvplot(
    x="Lng",
    y="Lat",
    hover_cols=["City", "Country", "Hotel Name", "Humidity"],
    geo=True,
    tiles="OSM",
    title="Hotel Locations by Humidity Levels in Global Cities",
    frame_width=500,
    frame_height=500,
    kind='points',
    color='city_norm',
    cmap='tab20',
    alpha=0.5,
    size="Humidity"
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (city_norm,Humidity,City,Country,Hotel Name)